In [ ]:
# 신경망 모델을 구성하기 위한 패키지 임포트

from keras import layers
from keras import models
from keras import optimizers
from keras import losses
from keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.utils import to_categorical

In [ ]:
# 정답 데이터를 one hot encoding 합니다. 
train_labels = to_categorical(y_train)
test_labels = to_categorical(y_test)
print(test_labels)

In [ ]:
# 완전연결계층으로 3층 신경망을 구성합니다. 
# dropout, L2가중치감소를 썼으나 데이터가 360개로 상대적으로 적어 오히려 가중치가 더 감소되어 사용하지 않기로 하였습니다.
from  tensorflow.keras.layers  import  Dense, BatchNormalization,  Dropout

tf.random.set_seed(777)
model = models.Sequential()
model.add(layers.Dense(100, activation = 'relu', input_shape = (61, )))  # 61개의 입력뉴런으로 flatten 시킨것을 입력받는다

model.add(layers.Dense(50, activation = 'relu'))

model.add(layers.Dense(3, activation = 'softmax'))
model.summary()

In [ ]:
# 모델 구성단계
best_model_weights = '/content/drive/MyDrive/model_04_3.h5'  # 모델이 저장될 위치와 이름 

# 학습 시킨 모델을 저장할 때 가장 좋은 모델을 알아서 자동으로 생성
checkpoint = ModelCheckpoint(
    best_model_weights,
    monitor='val_accuracy', 
    verbose=1,  
    save_best_only=True, 
    mode='max', 
    save_weights_only=False                             
)

estop = EarlyStopping(monitor = 'val_accuracy', patience = 20, verbose = 1) # 얼리스탑 기능
callbacks = [checkpoint, estop]

model.compile(optimizer='adam',
              loss=losses.categorical_crossentropy,
              metrics=['accuracy'])

In [ ]:
# 모델 학습단계
history = model.fit(
    X_train_features,  # 훈련 데이터 (60개의 mel값 + 1개의 zero_crossing_rate의 평균값 )
    train_labels,  # 원핫 인코딩한 정답
    validation_data=(X_test_features,test_labels),
    epochs = 200, 
    verbose = 1,
    callbacks=callbacks,
)

In [ ]:
# epochs에 따른 accuacy와 loss 선그래프
import matplotlib.pyplot as plt
print(history.history.keys())
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs = range(1, len(acc)+1)

ax1=plt.subplot(2,1,1)
plt.plot(epochs, acc, 'b', label = "training accuracy")
plt.plot(epochs, val_acc, 'r', label = "validation accuracy")
plt.ylabel('acc')
plt.xticks(visible=False)
plt.title('Training and validation accuracy')
plt.legend()

ax2=plt.subplot(2,1,2,sharex=ax1)
plt.plot(epochs, loss, 'b', label = "training loss")
plt.plot(epochs, val_loss, 'r', label = "validation loss")
plt.title('Training and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
np.where(y_test==0) # 0:새, 1:고양이, 2:개 테스트데이터에서 각 동물별 음성데이터 인덱스 번호 알기위해서

In [ ]:
# 테스트데이터 맞춰보기, 118번은 새소리 한번 맞춰보자
import IPython.display as ipd
nr_to_predict = 118 
pred = model.predict(X_test_features[nr_to_predict].reshape(1, 61,))
print("Bird: {} Cat: {} Dog: {}".format(pred[0][0], pred[0][1], pred[0][2]))
if (y_test[nr_to_predict] == 0):
    print ("This is a bird twitter")
elif (y_test[nr_to_predict] == 1):
    print ("This is a cat meowing")
elif (y_test[nr_to_predict] == 2):
    print ("This is a Dog barking")
    
plt.plot(X_test_features[nr_to_predict])
ipd.Audio(X_test[nr_to_predict],  rate=wav_rate)

In [ ]:
# 외부데이터 (고양이소리) 맞춰보기
audio_path='/content/drive/MyDrive/ColabNotebooks/sound/cat6.wav'

cat_y, cat_sr =librosa.load(audio_path) # 외부데이터 cat의 진폭, cat_y만 이용

# cat_test=librosa.read_wav_files(cat_test) # y만 담겼기때문에 할 필요 x 
cat_test_features=extract_features(cat_y,wav_rate)
# cat_test_features[0]

nr_to_predict = 0 # 외부데이터는 파일갯수 하나이기때문에 [0] 적용
pred = model.predict(cat_test_features[nr_to_predict].reshape(1, 61,))
print("Bird: {} Cat: {} Dog: {}".format(pred[0][0], pred[0][1], pred[0][2]))
a=[pred[0][0], pred[0][1], pred[0][2]]
b=np.array(a)

if (np.argmax(b) == 0):
    print ("This is a bird twitter")
elif (np.argmax(b) == 1):
    print ("This is a cat meowing")
elif (np.argmax(b) == 2):
    print ("This is a Dog barking")

plt.plot(cat_test_features[nr_to_predict])
import IPython.display as ipd
ipd.Audio(librosa.load(audio_path)[nr_to_predict],  rate=wav_rate)

In [ ]:
# 외부고양이 데이터 spectogram 보기
import librosa
from scipy import signal
freq_axis, t_axis,spec=signal.spectrogram(x=cat_y,fs=wav_rate,nperseg=220, noverlap=110,nfft=256)
plt.pcolormesh(t_axis,freq_axis,spec)
plt.ylabel('Frequency')
plt.xlabel('Time')
plt.title('spectrogram')
plt.show()